In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [3]:
ROOT = "/Users/kkalyan/github/airQo/"
train = pd.read_csv(ROOT+ "data/train-proc-v2.csv")
test = pd.read_csv(ROOT+ "data/test-proc-v2.csv")

In [4]:
pca_train = pd.read_csv(ROOT+ "data/datapca-train.csv")
pca_test = pd.read_csv(ROOT+ "data/datapca-test.csv")

In [7]:
train_all = pd.concat([pca_train, train], axis=1)
test_all = pd.concat([pca_test, test], axis=1)

In [8]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [9]:
lab.fit(train_all["location"])

LabelEncoder()

In [11]:
train_all.location  = lab.transform(train_all.location)
test_all.location  = lab.transform(test_all.location)

In [12]:
from catboost import CatBoostRegressor

In [13]:
X = train_all.drop(["ID",'target'],axis =1)
y = train_all.target

In [14]:
test.drop("target",axis  = 1,inplace  = True)

In [16]:
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators=7000, eval_metric='RMSE',learning_rate=0.1, random_seed= 1234, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test_all.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 41.2563085	test: 41.2563085	test1: 41.8793924	best: 41.8793924 (0)	total: 155ms	remaining: 18m 2s
100:	learn: 26.7633636	test: 26.7633636	test1: 28.9022107	best: 28.9022107 (100)	total: 6.99s	remaining: 7m 57s
200:	learn: 23.1438995	test: 23.1438995	test1: 26.9062646	best: 26.9062646 (200)	total: 14.3s	remaining: 8m 2s
300:	learn: 20.5644644	test: 20.5644644	test1: 26.1952654	best: 26.1952654 (300)	total: 20.6s	remaining: 7m 39s
400:	learn: 18.5993899	test: 18.5993899	test1: 25.6189973	best: 25.6183135 (399)	total: 26.9s	remaining: 7m 23s
500:	learn: 16.9558548	test: 16.9558548	test1: 25.3204783	best: 25.3135744 (499)	total: 33.3s	remaining: 7m 11s
600:	learn: 15.5406108	test: 15.5406108	test1: 25.0688290	best: 25.0639026 (598)	total: 39.7s	remaining: 7m 2s
700:	learn: 14.4412849	test: 14.4412849	test1: 24.8957496	best: 24.8939607 (699)	total: 46s	remaining: 6m 53s
800:	learn: 13.3676716	test: 13.3676716	test1: 24.6816191	best: 24.6816191 (800)	total: 52.5s	remaining: 6m 46s


24.13075402847982

In [18]:
from datetime import datetime
exp_time = str(datetime.now().replace(second=0, microsecond=0)).replace(" ", "_")
exp_name = "catboost-pca"
csv_file_name = "{}_{}".format(exp_time, exp_name)

In [19]:
test = pd.read_csv(ROOT+ "data/test-proc-v2.csv")
test_id  = test['ID']
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [20]:
SUB = "/Users/kkalyan/github/airQo/submissions/"

In [21]:
sub.to_csv(SUB+csv_file_name+'.csv', index=False)